In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv("keller_2016_unclean.csv")

# Step 1: Calculate average and standard deviation
numeric_columns = df.columns[df.columns.astype(str).str.isnumeric()]
df['Mean'] = df[numeric_columns].mean(axis=1)/100
df['StDev'] = df[numeric_columns].std(axis=1)/100
df['Median'] = df[numeric_columns].median(axis=1)/100
df['NonZeroMean'] = (df[numeric_columns] != 0).mean(axis=1)/100
df['NonZeroStDev'] = (df[numeric_columns] != 0).std(axis=1)/100
df['NonZeroProportion'] = df[numeric_columns].replace(0, np.nan).mean(axis=1)/100

# Step 2 and 3: Pivot and create statistics columns for each descriptor
pivot_df = df.pivot_table(
    values=['Mean', 'StDev', 'Median', 'NonZeroMean', 'NonZeroStDev', 'NonZeroProportion'],
    index=['Stimulus', 'Dilution'],
    columns='Descriptor',
    aggfunc='first'
)

# Flatten the column multi-index
pivot_df.columns = [f'{descriptor}_{stat.lower()}' for stat, descriptor in pivot_df.columns]

# Reset the index
pivot_df = pivot_df.reset_index()

# Merge with original dataframe
result_df = pd.merge(
    pivot_df, 
    df[['CID', 'IsomericSMILES', 'IUPACName', 'name']].drop_duplicates(subset='CID'), 
    left_on="Stimulus", 
    right_on="CID", 
    how='left'
)

result_df = result_df.loc[result_df.groupby('CID')['Dilution'].idxmax()]

# Reset the index if needed
result_df = result_df.reset_index(drop=True)

result_df.drop(columns=['Stimulus', 'Dilution'], inplace=True)
result_df.to_csv("keller_2016_unclean_processed.csv", index=False)

In [29]:
label_columns = result_df.columns[:-4].to_list()
file_df = pd.read_csv("../file_cleaning_features.csv")
file_df.loc[file_df['dataset'] == 'keller_2016', 'label_columns'] = str(label_columns)
file_df.to_csv("../file_cleaning_features.csv", index=False)